In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import warnings
import time

warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
    roc_curve,
    auc,
    roc_auc_score,
)
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [5]:
# loading data
X_train_balanced = np.load(
    "/home/samir-dahal/Diabetes_Classification/data/processed/X_train_balanced.npy"
)
y_train_balanced = np.load(
    "/home/samir-dahal/Diabetes_Classification/data/processed/y_train_balanced.npy"
)
x_test = pd.read_csv(
    "/home/samir-dahal/Diabetes_Classification/data/processed/X_test.csv"
)
y_test = pd.read_csv(
    "/home/samir-dahal/Diabetes_Classification/data/processed/y_test.csv"
)

In [7]:
# Initialize logestic regression
log_reg = LogisticRegression(
    solver="lbfgs", multi_class="ovr", max_iter=1000, random_state=42, n_jobs=-1
)

In [ ]:
#traning
start_time = time.time()
log_reg.fit(X_train_balanced, y_train_balanced)
training_time = time.time() - start_time
print(f"Training time is :{training_time:.2f}")

Training time is :1.81


In [ ]:
#Prediction
y_pred_lr = log_reg.predict(x_test)
y_pred_proba_lr = log_reg.predict_proba(x_test)


In [ ]:
#Evaluate logestic regression

# Overall accuracy
accuracy_lr = accuracy_score(y_test,y_pred_lr)
print(f"Over all accuracy{accuracy_lr}")

precision_lr = precision_score(y_test, y_pred_lr, average=None, zero_division=0)
recall_lr = recall_score(y_test, y_pred_lr, average=None, zero_division=0)
f1_lr = f1_score(y_test, y_pred_lr, average=None, zero_division=0)

precision_macro_lr = precision_score(
    y_test, y_pred_lr, average="macro", zero_division=0
)
recall_macro_lr = recall_score(y_test, y_pred_lr, average="macro", zero_division=0)
f1_macro_lr = f1_score(y_test, y_pred_lr, average="macro", zero_division=0)



Over all accuracy0.9481865284974094
